#### Importowanie modułów

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms, datasets
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Zbiory danych, modele, metody TL, hiperparametry

In [17]:
DATASETS = [
    'CIFAR10'
]
MODELS = [
    #models.alexnet(pretrained=True),
    models.vgg16(pretrained=True),
    models.resnet50(pretrained=True)
]
TL_METHODS = [
    #'no_freezing',
    'classic_tl',
    #'fine_tuning'
]

# Hyperparmeters
EPOCHS = 10
BATCH_SIZE = 64

#### Utworzenie Data setów i data loaderów

In [18]:
def create_datasets_and_dataloaders(dataset_name: str, model):

    if isinstance(model, models.AlexNet) or isinstance(model, models.VGG) or isinstance(model, models.ResNet):
        base_transforms = [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    else:
        base_transforms = [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]

    train_transform = transforms.Compose(base_transforms)
    test_transform = transforms.Compose(base_transforms)

    preprocessing = [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ]

    train_transform.transforms.extend(preprocessing)

    if dataset_name == 'CIFAR10':
        train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

        test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
        test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)
        print("Loaders created")

    return train_set, test_set, train_loader, test_loader

#### Przetwarzanie modeli do transfer learningu

In [19]:
def apply_transfer_learning(model, tl_method):
    if tl_method == 'classic_tl':
        for param in model.parameters():
            param.requires_grad = False
        if hasattr(model, 'classifier'):
            for param in model.classifier.parameters():
                param.requires_grad = True
        elif hasattr(model, 'fc'):
            for param in model.fc.parameters():
                param.requires_grad = True
    elif tl_method == 'fine_tuning' or tl_method == 'no_freezing':
            for param in model.parameters():
                param.requires_grad = True



def customize_model(model, num_classes, tl_method):
    if isinstance(model, models.AlexNet):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.VGG):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    print("Classifier for model has been changed.")
    apply_transfer_learning(model, tl_method)
    print("Transfer learning method:", tl_method, "applied.")

    return model

#### Funkcja mrożąca do naszej metody

In [20]:

def freeze_percentage(model, percentage):
    """Freeze a percentage of the model's parameters based on gradient norms."""
    grad_params = [(param.grad.norm(), param) for param in model.parameters() if param.requires_grad and param.grad is not None]
    grad_params.sort(key=lambda x: x[0])
    num_params_to_freeze = int(percentage * len(grad_params))

    for _, param in grad_params[:num_params_to_freeze]:
        param.requires_grad = False

#### Ustawienia fine-tuningu

In [21]:
def adjust_fine_tuning(model, optimizer):
    """Adjust the model and optimizer for fine-tuning."""
    for param in model.parameters():
        param.requires_grad = True
    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.0001

#### Pętla ucząco walidacyjna

In [22]:
# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using: {device}")

def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, tl_method):
    """Train and evaluate the model."""

    if tl_method == 'fine_tuning':
        adjust_fine_tuning(model, optimizer)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        with tqdm(train_loader, desc=f"Epoch {epoch + 1}") as tepoch:
            for images, labels in tepoch:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                tepoch.set_postfix(loss=loss.item())

        evaluate_model(model, test_loader, criterion, running_loss, tl_method, optimizer)

def evaluate_model(model, test_loader, criterion, running_loss, tl_method, optimizer):
    """Evaluate the model and adjust learning rate if fine-tuning."""
    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Train Loss: {running_loss/len(train_loader)}, '
          f'Validation Loss: {val_loss/len(test_loader)}, '
          f'Accuracy: {100 * correct / total}%')


# Main loop
for id_dataset, dataset in enumerate(DATASETS):
    print(f"Using the {dataset} dataset.")
    

    for id_tl_method, tl_method in enumerate(TL_METHODS):
        print(f"Using the {tl_method} method")

        for id_model, model in enumerate(MODELS):
            train_set, test_set, train_loader, test_loader = create_datasets_and_dataloaders(dataset, model=model)
            num_classes = len(train_set.classes)
            
            print(f"Using the {model.__class__.__name__} model")
            model = customize_model(model, num_classes, tl_method)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            model.to(device)

            print("Training starts")
            train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, EPOCHS, tl_method)


Using: cuda
Using the CIFAR10 dataset.
Using the classic_tl method
Files already downloaded and verified
Files already downloaded and verified
Loaders created
Using the VGG model
Classifier for model has been changed.
Transfer learning method: classic_tl applied.
Training starts


Epoch 1:   0%|          | 0/782 [00:12<?, ?it/s]


KeyboardInterrupt: 